In [1]:
!pip install polars numpy implicit tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 10.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 9.5 MB/s eta 0:00:0000:01:00:01m


In [1]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit

RANDOM_STATE = 42
N_PREDICTIONS = 100


/home/jovyan/shares/SR004.nfs2/amaksimova/env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [2]:
train = pl.read_parquet("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/hh_recsys_train_hh.pq")
test = pl.read_parquet("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/hh_recsys_test_hh.pq")
vacancies = pl.read_parquet("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/hh_recsys_vacancies.pq")

In [3]:
train = train.to_pandas()
test = test.to_pandas()
vacancies = vacancies.to_pandas()

In [94]:
name = vacancies['name']

In [ ]:
/ // \ () 

In [95]:
import re
def parenthesis_processing(s: str):
    pattern1 = r"\((.*?)\)"
    remainder = re.findall(pattern1, s)
    s_new = re.sub(pattern1, "", s)
    pattern2 = r"\s+"
    result = re.sub(pattern2, " ", s_new)
    result = result.strip()
    return result, remainder



In [96]:
#test
s = 'Контент-маркетолог / копирайтер (Г. Ижевск)'
print(parenthesis_processing(s))
s = 'Контент-маркетолог / копирайтер'
print(parenthesis_processing(s))
s = 'Контент-маркетолог / копирайтер (Г. Ижевск) (Удаленная работа)'
print(parenthesis_processing(s))

('Контент-маркетолог / копирайтер', ['Г. Ижевск'])
('Контент-маркетолог / копирайтер', [])
('Контент-маркетолог / копирайтер', ['Г. Ижевск', 'Удаленная работа'])


In [97]:
name_no_parenthesis = []
parenthesis = []
for s in name:
    result, remainder = parenthesis_processing(s)
    name_no_parenthesis.append(result)
    parenthesis.append(remainder)

In [98]:
print(f"всего уникальных имен {len(name_no_parenthesis)}, уникальных имен без скобочек {len(set(name_no_parenthesis))}")

всего уникальных имен 2734129, уникальных имен без скобочек 535320


In [99]:
import re
def slash_processing(s: str):
    parts = [i.strip() for i in re.split(r'/|//|\\|\\\\', s) if len(i.strip())>0]
    if len(parts)>0:
        return parts[0], parts[1:]
    else:
        print(s,'пусто')
        return "",[]

In [100]:
#test
s = 'Контент-маркетолог / копирайтер'
print(slash_processing(s))
s = 'Контент-маркетолог / копирайтер\\ спанч боб'
print(slash_processing(s))
s = 'Контент-маркетолог //// копирайтер \штаны'
print(slash_processing(s))
s = 'Контент-маркетолог '
print(slash_processing(s))

('Контент-маркетолог', ['копирайтер'])
('Контент-маркетолог', ['копирайтер', 'спанч боб'])
('Контент-маркетолог', ['копирайтер', 'штаны'])
('Контент-маркетолог', [])


In [101]:
name_no_parenthesis_no_slash = []
slash = []
for s in name_no_parenthesis:
    result, remainder = slash_processing(s)
    name_no_parenthesis_no_slash.append(result)
    slash.append(remainder)

 пусто
 пусто


In [102]:
print(f"всего уникальных имен {len(name_no_parenthesis)}, уникальных имен без скобочек и слэшей {len(set(name_no_parenthesis_no_slash))}")

всего уникальных имен 2734129, уникальных имен без скобочек и слэшей 441635


In [103]:
import itertools
name_union = name_no_parenthesis_no_slash+list(itertools.chain.from_iterable(slash))

In [104]:
from collections import Counter
name_c = Counter(name_union)

In [105]:
c = 0
for i in range(len(name_no_parenthesis_no_slash)):
    for j in range(len(slash[i])):
        if name_c[name_no_parenthesis_no_slash[i]]<name_c[slash[i][j]]:
            name_no_parenthesis_no_slash[i],slash[i][j] = slash[i][j],name_no_parenthesis_no_slash[i]
            c+=1
print(c)

63180


In [106]:
print(f"уникальных имен с выбором по слешу популярных вакансий {len(set(name_no_parenthesis_no_slash))}")

уникальных имен с выбором по слешу популярных вакансий 432967


In [107]:
len(parenthesis)

2734129

In [108]:
vacancies['clean_names'] = name_no_parenthesis_no_slash
vacancies['slash_names'] = slash
vacancies['parenthesis'] = parenthesis

In [112]:
import pandas as pd
names_ds = pd.DataFrame()
names_ds['clean_names'] = name_no_parenthesis_no_slash
names_ds['slash_names'] = slash
names_ds['parenthesis'] = parenthesis
names_ds.to_json(
        "/home/jovyan/shares/SR004.nfs2/amaksimova/exp/names.jsonl", orient="records", force_ascii=False
    )

In [109]:
vacancies

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,clean_names,slash_names,parenthesis
0,v_862116,Смотритель музейный,c_162972,<strong>Обязанности:</strong> <ul> <li>Осущест...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,[],[]
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"<p><strong>Возможно, наша вакансия именно та, ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,[],[]
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,<strong>Обязанности:</strong> <ul> <li>Расчет ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,[],[по расчету зарплаты]
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,"<p><strong>Для каждого, кто хочет работать и з...",None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,[],"[Токсово, Привокзальная, 16]"
4,v_312507,Торговый представитель (г. Абакан),c_206699,<p>Компания ТД &quot;Оливьера&quot;- лучший ди...,"[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,[],[г. Абакан]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734124,v_2484959,Сборщик-упаковщик,c_203256,<p><strong>Обязанности:</strong></p> <ul> <li>...,"[Пользователь ПК, Умение работать в команде, У...",40000.0,90000.0,RUR,a_5387,ar_71,full,flexible,noExperience,Сборщик-упаковщик,[],[]
2734125,v_205163,Сварщик на полуавтомат,c_158695,<p>ОКЛАД 80 000 РУБЛЕЙ. В связи с увеличением ...,"[Желание работать и зарабатывать, Высокая энер...",80000.0,130000.0,RUR,a_5527,ar_69,full,fullDay,between1And3,Сварщик на полуавтомат,[],[]
2734126,v_639897,Главный инженер / Технический директор,c_209365,<p>Компания<strong> МОБИЛЬНЫЙ ДОМ - ЛИДЕР В СФ...,"[Контроль исправности оборудования, Инженерные...",200000.0,NaN,RUR,a_1756,ar_41,full,fullDay,between1And3,Главный инженер,[Технический директор],[]
2734127,v_1636531,"Провизор/Фармацевт (г.Адыгейск, 20 км от Красн...",c_246244,<strong>Требования:</strong> <ul> <li> <p>Зако...,"[Предпечатная подготовка, Статистический анализ]",25000.0,NaN,RUR,a_3403,ar_60,full,fullDay,noExperience,Фармацевт,[Провизор],"[г.Адыгейск, 20 км от Краснодара]"


In [5]:
train

,user_id,session_id,vacancy_id,action_type,action_dt
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[2023-11-01T00:40:58.105000000, 2023-11-01T00:..."
1,u_1057881,s_33868982,[v_665861],[2],[2023-11-01T00:23:51.452000000]
2,u_1036784,s_32474802,[v_2594840],[2],[2023-11-01T00:52:34.023000000]
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[2023-11-01T00:58:20.793000000, 2023-11-01T01:..."
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[2023-11-01T01:14:20.828000000, 2023-11-01T00:..."
...,...,...,...,...,...
3463059,u_202578,s_8306993,[v_1499243],[2],[2023-11-13T21:01:41.813000000]
3463060,u_895531,s_7293998,[v_2041381],[2],[2023-11-14T17:51:22.169000000]
3463061,u_317562,s_9866576,[v_503192],[2],[2023-11-14T20:03:54.988000000]
3463062,u_225581,s_19724962,[v_2526106],[2],[2023-11-14T20:52:51.985000000]


In [6]:
test

,user_id,session_id,vacancy_id,action_type,action_dt
0,u_482520,s_25018731,"[v_2597196, v_1223061, v_1223061]","[2, 2, 1]","[2023-11-19T12:03:13.089000000, 2023-11-19T12:..."
1,u_582132,s_481216,"[v_470400, v_470400, v_1530783]","[2, 1, 2]","[2023-11-21T15:39:47.981000000, 2023-11-21T15:..."
2,u_212584,s_16918781,"[v_1572055, v_1572055, v_1572055, v_953153, v_...","[2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1]","[2023-11-16T08:41:47.031000000, 2023-11-16T08:..."
3,u_425177,s_17505104,"[v_1375331, v_1922852]","[2, 2]","[2023-11-17T12:42:18.513000000, 2023-11-17T12:..."
4,u_700997,s_15528830,"[v_2152997, v_2152997, v_1217630]","[2, 1, 2]","[2023-11-16T17:22:53.530000000, 2023-11-16T17:..."
...,...,...,...,...,...
83184,u_499368,s_19158589,"[v_1716634, v_2232327, v_2232327, v_2232327, v...","[2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2]","[2023-11-18T03:15:15.530000000, 2023-11-18T03:..."
83185,u_1084868,s_24756162,"[v_395706, v_2405231]","[2, 2]","[2023-11-15T18:06:16.550000000, 2023-11-15T18:..."
83186,u_1138032,s_25291467,"[v_1213925, v_1213925, v_2359832, v_2359832]","[2, 1, 2, 1]","[2023-11-19T23:19:44.092000000, 2023-11-19T23:..."
83187,u_608102,s_18813020,"[v_933773, v_933773, v_2021127, v_1508520, v_2...","[3, 2, 2, 2, 2, 1, 2]","[2023-11-16T15:40:09.408000000, 2023-11-16T15:..."


In [22]:
vacancies

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,list[str],i64,i64,str,str,str,str,str,str
"""v_862116""","""Смотритель муз…","""c_162972""","""<strong>Обязан…","[""Пользователь ПК"", ""Работа в команде"", … ""PR-консультирование""]",16500,null,"""RUR""","""a_4761""","""ar_33""","""full""","""fullDay""","""noExperience"""
"""v_288642""","""Ведущий менедж…","""c_208672""","""<p><strong>Воз…","[""Активные продажи"", ""Холодные продажи"", … ""Организация мероприятий""]",50000,null,"""RUR""","""a_744""","""ar_2""","""full""","""fullDay""","""noExperience"""
"""v_1840054""","""Бухгалтер (по …","""c_198109""","""<strong>Обязан…",null,50000,65000,"""RUR""","""a_6223""","""ar_78""","""full""","""fullDay""","""between3And6"""
"""v_2346232""","""Пекарь (Токсов…","""c_6137""","""<p><strong>Для…",null,38500,42000,"""RUR""","""a_4795""","""ar_51""","""full""","""fullDay""","""noExperience"""
"""v_312507""","""Торговый предс…","""c_206699""","""<p>Компания ТД…","[""Продуктивность"", ""Клиентоориентированность"", … ""Развитие продаж""]",60000,null,"""RUR""","""a_6837""","""ar_4""","""full""","""fullDay""","""between1And3"""
"""v_2323706""","""Менеджер по ка…","""c_244141""","""<p><strong>X</…","[""Контроль качества"", ""Производственный контроль"", … ""Лабораторная диагностика""]",88000,null,"""RUR""","""a_5920""","""ar_64""","""full""","""fullDay""","""between1And3"""
"""v_704266""","""Менеджер по пр…","""c_192038""","""<p>Если ты ище…","[""Активные продажи"", ""Поиск и привлечение клиентов"", … ""Проведение переговоров""]",30000,100000,"""RUR""","""a_5398""","""ar_71""","""full""","""fullDay""","""between1And3"""
"""v_525733""","""Врач по ЛФК""","""c_5101""","""<strong>Обязан…",null,null,null,null,"""a_5043""","""ar_96""","""full""","""fullDay""","""between3And6"""
"""v_2091086""","""Монтажник СТиЖ…","""c_230890""","""<em><strong>Ст…",null,100000,null,"""RUR""","""a_500""","""ar_36""","""full""","""flyInFlyOut""","""between1And3"""


In [1]:
import pandas as pd
name_ds = pd.read_json("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/names.jsonl")

In [2]:
vacancies_ds = pd.read_json("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/hh_recsys_vacancies.jsonl")

In [5]:
name_ds['name'] = vacancies_ds['name']
name_ds.to_json(
        "/home/jovyan/shares/SR004.nfs2/amaksimova/exp/names.jsonl", orient="records", force_ascii=False
    )